In [ ]:
import ee
!earthengine authenticate
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=b0_lU7RCirWlwMHR9EBu-pdZwXW43tkCzWSmKqJ9bfE&tc=8Mywas_Q75T--K_YrKI-KRTZv6CIZJDHI1qitV_aBaQ&cc=dnQ7gYueMBo3eYuggXCTt_veLvtfEy5AJlPJ4gUH0wc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qg2mM_gg9xWVpb5u09WDzRbzb8iNC9xFvxIf9l0D0bkTOxggUT-NhY

Successfully saved authorization token.


In [ ]:
from google.colab import drive
import os
dir = '/content/drive/'
drive.mount(dir)

datapath = 'Research/Weather Risk/Data/'

Mounted at /content/drive/


In [ ]:
#this is the part we need to change!!
parish_boundaries = ee.FeatureCollection("projects/ee-sprout-protect/assets/GADM_KE")
#landcover data
#modisLandcover = ee.ImageCollection("MODIS/006/MCD12Q1")
# This data is from https://www.usgs.gov/centers/western-geographic-science-center/science/global-food-security-support-analysis-data-30-m
# Will need to download the tiles for new countries and then import as assets in EE
updated_landcover = ee.Image("projects/ee-ahobbs/assets/GFSAD_Africa");


#"projects/ee-sprout-protect/assets/GADM_KE"

modis = ee.ImageCollection("MODIS/006/MOD09GA")

#NDVI data
ndvi = ee.ImageCollection("LANDSAT/LE07/C01/T1_8DAY_NDVI")

#Data on soil moisture, temp, humidity, wind speed etc.
soil_moisture = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture").select("ssm") #data available after 2014

#GDAL before 2010 (including 2010)
GDAL = ee.ImageCollection("NASA/GLDAS/V20/NOAH/G025/T3H").select( "RootMoist_inst","Qs_acc","Evap_tavg","Qair_f_inst","Psurf_f_inst",
                                                                                                      "Wind_f_inst", "Tveg_tavg", "Tair_f_inst")

#GDAL after 2010 (2011 - 2020)
GDAL_new = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H").select( "RootMoist_inst","Qs_acc","Evap_tavg","Qair_f_inst","Psurf_f_inst",
                                                                                                      "Wind_f_inst", "Tveg_tavg", "Tair_f_inst")
temp_precip_wind = ee.ImageCollection("ECMWF/ERA5/DAILY").select("mean_2m_air_temperature", "total_precipitation")
#temperature = ee.ImageCollection('MODIS/006/MOD11A1') #gives only temp data


In [ ]:
# USE THIS CELL FOR EXTRACTING MODIS DATA WITH CROPLAND FILTER:


# extracting monthly data parish level data: BANDS NEEDED TO CALCULATE VEGETATION INDICES
infos = []
for year in range(2011, 2021):
  #filtered = modisLandcover.filter(ee.Filter.date('{}-01-01'.format(year), '{}-12-31'.format(year)))
  #landcover = ee.Image(filtered.first())
  #classified = landcover.select('LC_Type1')
  # ugandaLandcover = classified.clip(parish_boundaries)
  # croplands = ugandaLandcover.eq([12,14]).reduce(ee.Reducer.anyNonZero())

  filtered = updated_landcover
  landcover = ee.Image(filtered)
  classified = landcover.select('b1')
  ugandaLandcover = classified.clip(parish_boundaries)
  croplands = ugandaLandcover.eq(2).reduce(ee.Reducer.anyNonZero())

  for month in range(1,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"
    if month in [1,3,5,7,8,10,12]:
      lastday = "-31"
    elif month == 2:
      lastday = "-28"
    else:
      lastday = "-30"

    end_date = str(year) + "-" + str(month) + lastday
    #print(start_date, end_date)
    modis_avg = modis.filter(ee.Filter.date(start_date, end_date)).max().clip(parish_boundaries).mask(croplands)

    meanMODIS = modis_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanMODIS,
      folder= "MODIS_croplands_kenya",
      description='kenya' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()

In [ ]:
#!earthengine task list

In [ ]:
# USE THIS CELL FOR EXTRACTING MODIS DATA WITHOUT CROPLAND FILTER:

'''
# extracting monthly data parish level data: BANDS NEEDED TO CALCULATE VEGETATION INDICES
infos = []
for year in range(2004, 2021):

  for month in range(5,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"
    end_date = str(year) + "-" + str(month) + "-30"
    #print(start_date, end_date)
    modis_avg = modis.filter(ee.Filter.date(start_date, end_date)).max()

    meanMODIS = modis_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanMODIS,
      folder= "MODIS",
      description='uganda' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()
'''

In [ ]:
# USE THIS CELL FOR EXTRACTING NDVI DATA WITH CROPLAND FILTER:


# extracting monthly data parish level data: NDVI
infos = []
for year in range(2011, 2021):
  # filtered = modisLandcover.filter(ee.Filter.date('{}-01-01'.format(year), '{}-12-31'.format(year)))
  # landcover = ee.Image(filtered.first())
  # classified = landcover.select('LC_Type1')
  # ugandaLandcover = classified.clip(parish_boundaries)
  # croplands = ugandaLandcover.eq([12,14]).reduce(ee.Reducer.anyNonZero())

  filtered = updated_landcover
  landcover = ee.Image(filtered)
  classified = landcover.select('b1')
  ugandaLandcover = classified.clip(parish_boundaries)
  croplands = ugandaLandcover.eq(2).reduce(ee.Reducer.anyNonZero())

  for month in range(1,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"

    if month in [1,3,5,7,8,10,12]:
      lastday = "-31"
    elif month == 2:
      lastday = "-28"
    else:
      lastday = "-30"

    end_date = str(year) + "-" + str(month) + lastday
    #print(start_date, end_date)
    ndvi_avg = ndvi.filter(ee.Filter.date(start_date, end_date)).max().clip(parish_boundaries).mask(croplands)

    meanNDVI = ndvi_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanNDVI,
      folder= "NDVI_croplands_kenya",
      description='Keinya' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()

In [ ]:
# USE THIS CELL FOR EXTRACTING NDVI DATA WITHOUT CROPLAND FILTER:

'''
# extracting monthly data parish level data: NDVI
infos = []
for year in range(2004, 2021):
  for month in range(5,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"
    end_date = str(year) + "-" + str(month) + "-30"
    #print(start_date, end_date)
    ndvi_avg = ndvi.filter(ee.Filter.date(start_date, end_date)).max()

    meanNDVI = ndvi_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanNDVI,
      folder= "NDVI",
      description='uganda' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()
  '''

In [ ]:
# USE THIS CELL FOR EXTRACTING TPW DATA WITH CROPLAND FILTER:


# extracting monthly data parish level data: for all other related variables such as precipitation, wind, temperature
infos = []
for year in range(2011, 2021):
  # filtered = modisLandcover.filter(ee.Filter.date('{}-01-01'.format(year), '{}-12-31'.format(year)))
  # landcover = ee.Image(filtered.first())
  # classified = landcover.select('LC_Type1')
  # ugandaLandcover = classified.clip(parish_boundaries)
  # croplands = ugandaLandcover.eq([12,14]).reduce(ee.Reducer.anyNonZero())

  filtered = updated_landcover
  landcover = ee.Image(filtered)
  classified = landcover.select('b1')
  ugandaLandcover = classified.clip(parish_boundaries)
  croplands = ugandaLandcover.eq(2).reduce(ee.Reducer.anyNonZero())

  for month in range(1,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"

    if month in [1,3,5,7,8,10,12]:
      lastday = "-31"
    elif month == 2:
      lastday = "-28"
    else:
      lastday = "-30"

    end_date = str(year) + "-" + str(month) + lastday

    #print(start_date, end_date)
    temp_precip_wind_avg = temp_precip_wind.filter(ee.Filter.date(start_date, end_date)).mean().clip(parish_boundaries).mask(croplands)

    meanTPW = temp_precip_wind_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanTPW,
      folder= "Temp_Precip_Wind_croplands_kenya",
      description='kenya' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()

In [ ]:
# USE THIS CELL FOR EXTRACTING TPW DATA WITH CROPLAND FILTER:

'''
# extracting monthly data parish level data: for all other related variables such as precipitation, wind, temperature
infos = []
for year in range(2004, 2021):
  for month in range(5,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"
    end_date = str(year) + "-" + str(month) + "-30"
    #print(start_date, end_date)
    temp_precip_wind_avg = temp_precip_wind.filter(ee.Filter.date(start_date, end_date)).mean()

    meanTPW = temp_precip_wind_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanTPW,
      folder= "Temp_Precip_Wind",
      description='uganda' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()
'''

In [ ]:
# USE THIS CELL FOR EXTRACTING GDAL DATA WITH CROPLAND FILTER: 2004 - 2010


# extracting monthly data parish level data: GDAL (GDAL contains data for important variables such as precipitation, temprature, wind speed, transpiration, soil moisture etc.) [2004 - 2010]
infos = []
for year in range(2011, 2021):
  # filtered = modisLandcover.filter(ee.Filter.date('{}-01-01'.format(year), '{}-12-31'.format(year)))
  # landcover = ee.Image(filtered.first())
  # classified = landcover.select('LC_Type1')
  # ugandaLandcover = classified.clip(parish_boundaries)
  # croplands = ugandaLandcover.eq([12,14]).reduce(ee.Reducer.anyNonZero())

  filtered = updated_landcover
  landcover = ee.Image(filtered)
  classified = landcover.select('b1')
  ugandaLandcover = classified.clip(parish_boundaries)
  croplands = ugandaLandcover.eq(2).reduce(ee.Reducer.anyNonZero())

  for month in range(1,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"

    if month in [1,3,5,7,8,10,12]:
      lastday = "-31"
    elif month == 2:
      lastday = "-28"
    else:
      lastday = "-30"

    end_date = str(year) + "-" + str(month) + lastday
    #print(start_date, end_date)
    GDAL_avg = GDAL.filter(ee.Filter.date(start_date, end_date)).mean().clip(parish_boundaries).mask(croplands)

    meanGDAL = GDAL_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanGDAL,
      folder= "GDAL_croplands_kenya",
      description='kenya' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()


In [ ]:
# USE THIS CELL FOR EXTRACTING GDAL DATA WITH CROPLAND FILTER: 2011 - 2020

# extracting monthly data parish level data: GDAL (GDAL contains data for important variables such as precipitation, temprature, wind speed, transpiration, soil moisture etc.) [2011 - 2020]
infos = []
for year in range(2011, 2021):
  # filtered = modisLandcover.filter(ee.Filter.date('{}-01-01'.format(year), '{}-12-31'.format(year)))
  # landcover = ee.Image(filtered.first())
  # classified = landcover.select('LC_Type1')
  # ugandaLandcover = classified.clip(parish_boundaries)
  # croplands = ugandaLandcover.eq([12,14]).reduce(ee.Reducer.anyNonZero())

  filtered = updated_landcover
  landcover = ee.Image(filtered)
  classified = landcover.select('b1')
  ugandaLandcover = classified.clip(parish_boundaries)
  croplands = ugandaLandcover.eq(2).reduce(ee.Reducer.anyNonZero())

  for month in range(1,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"

    if month in [1,3,5,7,8,10,12]:
      lastday = "-31"
    elif month == 2:
      lastday = "-28"
    else:
      lastday = "-30"

    end_date = str(year) + "-" + str(month) + lastday
    #print(start_date, end_date)
    GDAL_avg = GDAL_new.filter(ee.Filter.date(start_date, end_date)).mean().clip(parish_boundaries).mask(croplands)

    meanGDAL = GDAL_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanGDAL,
      folder= "GDAL_croplands_kenya",
      description='kenya' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()

In [ ]:
# USE THIS CELL FOR EXTRACTING GDAL DATA WITHOUT CROPLAND FILTER: 2004 - 2010

'''
# extracting monthly data parish level data: GDAL (GDAL contains data for important variables such as precipitation, temprature, wind speed, transpiration, soil moisture etc.) [2004 - 2010]
infos = []
for year in range(2004, 2011):
  for month in range(5,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"
    end_date = str(year) + "-" + str(month) + "-30"
    #print(start_date, end_date)
    GDAL_avg = GDAL.filter(ee.Filter.date(start_date, end_date)).mean()

    meanGDAL = GDAL_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanGDAL,
      folder= "GDAL",
      description='uganda' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()
'''

In [ ]:
# USE THIS CELL FOR EXTRACTING GDAL DATA WITHOUT CROPLAND FILTER: 2011-2020
'''
# extracting monthly data parish level data: GDAL (GDAL contains data for important variables such as precipitation, temprature, wind speed, transpiration, soil moisture etc.) [2011 - 2020]
infos = []
for year in range(2011, 2021):
  for month in range(5,13):
    print(year, month)
    start_date = str(year) + "-" + str(month) + "-01"
    end_date = str(year) + "-" + str(month) + "-30"
    #print(start_date, end_date)
    GDAL_avg = GDAL_new.filter(ee.Filter.date(start_date, end_date)).mean()

    meanGDAL = GDAL_avg.reduceRegions(
      collection=parish_boundaries,
      reducer=ee.Reducer.mean(),
      scale=30)

    #all remote sensing files are stored in data/geodata/Parish_level_monthly_data/
    task = ee.batch.Export.table.toDrive(
      collection=meanGDAL,
      folder= "GDAL",
      description='uganda' +  "_" + str(year) + "_" + str(month),
      fileFormat='CSV')
    task.start()

'''


In [ ]:
# extracting yearly data county/ district level data
'''
infos = []
for year in range(2004, 2021):
  print(year)
  ndvi_avg = ndvi.filter(ee.Filter.calendarRange(year, year,'year')).mean()
  #update: changed max to mean NDVI
  meanNDVI = ndvi_avg.reduceRegions(
    collection=boundaries,
    reducer=ee.Reducer.mean(),
    scale=30)
  task = ee.batch.Export.table.toDrive(
    collection=meanNDVI,
    folder= "County_District_level_yearly_data",
    description='uganda' + str(year),
    fileFormat='CSV')
  task.start()
  '''